# Introduction to Astrometry

### Dora Föhring, University of Hawaii Institute for Astronomy

Aim: To measure the position and uncertainty of a Near Earth Asteroid

## 0. Prerequisites

pip install astroquery

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as fits
from astropy import wcs
from astroquery.gaia import Gaia
import pdb
## make matplotlib appear in the notebook rather than in a new window
%matplotlib inline

### 0.1 Directory Set up & Display Image

In [ ]:
datadir = '/Users/dorafohring/Desktop/2016HO3/'
objname  = '2016HO3'

In [ ]:
def plotfits(imno):
    img = fits.open(datadir+objname+'_{0:02d}.fits'.format(numb))[0].data

    f = plt.figure(figsize=(10,12))
    im = plt.imshow(img, cmap='hot')
    im = plt.imshow(img[480:560, 460:540], cmap='hot')
    plt.clim(1800, 2800)
    plt.colorbar(im, fraction=0.046, pad=0.04)
    plt.savefig("figure{0}.png".format(imno))
    plt.show()

In [ ]:
numb = 1 
plotfits(numb)

## 1. Centroiding on Images

Write a text file with image centers.
Write code to open each image and extract centroid position from previous exercise.
Save results in a text file.

In [ ]:
centers = np.array([[502,501], [502,501]])
np.savetxt('centers.txt', centers, fmt='%i')
centers = np.loadtxt('centers.txt', dtype='int')

In [ ]:
searchr = 5

### 1.1 Center of Mass

In [ ]:
def cent_weight(n):
    """
    Assigns centroid weights
    """
    wghts=np.zeros((n),np.float)
    for i in range(n):
        wghts[i]=float(i-n/2)+0.5
    return wghts

def calc_CoM(psf, weights):
    """
    Finds Center of Mass of image
    """
    cent=np.zeros((2),np.float)
    temp=sum(sum(psf))
    cent[0]=sum(sum(psf)*weights)/temp
    cent[1]=sum(sum(psf.T)*weights)/temp
    return cent

In [ ]:
centlist = []
for i, center in enumerate(centers):
    image = fits.open(datadir+objname+'_{0:02d}.fits'.format(i+1))[0].data
    searchbox = image[center[0]-searchr : center[0]+searchr, center[1]-searchr : center[1]+searchr]
    boxlen = len(searchbox)
    weights = cent_weight(boxlen)
    cen_offset = calc_CoM(searchbox, weights)
    centlist.append(center + cen_offset)

In [ ]:
print(centlist[0])

## 2. Identifying Stars in the Field

### 2.1. Image Maxima

## Ex. 1 Locate the Maxima in the image

## 3. Converting pixel coordinates to WCS

In [ ]:
def load_wcs_from_file(filename, xx, yy):
    # Load the FITS hdulist using astropy.io.fits
    hdulist = fits.open(filename)

    # Parse the WCS keywords in the primary HDU
    w = wcs.WCS(hdulist[0].header)

    # Print out the "name" of the WCS, as defined in the FITS header
    print(w.wcs.name)

    # Print out all of the settings that were parsed from the header
    w.wcs.print_contents()

    # Three pixel coordinates of interest.
    # Note we've silently assumed a NAXIS=2 image here
    targcrd = np.array([centlist[0]], np.float_)
    
    starscrd = np.array([xx, yy], np.float_)

    # Convert pixel coordinates to world coordinates
    # The second argument is "origin".
    tworld = w.wcs_pix2world(targcrd, 0)
    sworld = w.wcs_pix2world(starscrd.T, 0)
    print(tworld)
    
    return w, sworld

Find position of Asteroid in WCS

In [ ]:
wparams, scoords = load_wcs_from_file(datadir+objname+'_{0:02d}.fits'.format(1), x2, y2)

In [ ]:
print(scoords)

## 3. Matching

### 3.1 Get astrometric catalog

In [ ]:
job = Gaia.launch_job_async("SELECT * \
FROM gaiadr1.gaia_source \
WHERE CONTAINS(POINT('ICRS',gaiadr1.gaia_source.ra,gaiadr1.gaia_source.dec),CIRCLE('ICRS', 193.34, 33.86, 0.08))=1;" \
, dump_to_file=True)

print (job)

In [ ]:
r = job.get_results()
print (r['source_id'], r['ra'], r['dec'])
print(type(r['ra']))

Convert Gaia WCS coordinates to pixels

## 3.2 Perform Match

In [ ]:
ra  = np.array(r['ra'])
dec = np.array(r['dec'])

xpix, ypix = wparams.wcs_world2pix(ra, dec, 0)

## Ex. 2 Find a way to match
E.g. Convert pixel coordinates to positions on grid and perform a cross-correlation.


## 3.3 Shift

## Ex. 3 Perform shift

Convert shifted coordinate into WCS

In [ ]:
wparams, scoords = load_wcs_from_file(datadir+objname+'_{0:02d}.fits'.format(1), targshifted[0], targshifted[1])